In [ ]:
from models_impute import *
from pygrinder import block_missing

Instructions for updating:
non-resource variables are not supported in the long term


d:\conda\envs\data\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



In [8]:
data = pd.read_csv("./data/lorenz/lorenz_dataset_0_timeseries.csv",header=None)
data = data.values

In [9]:
cg = pd.read_csv('./causality_matrices/lorenz_causality_matrix.csv', header=None)
cg = cg.values
model_params = {
        'num_levels': 10,
        'kernel_size': 8,
        'dilation_c': 2
    }

In [10]:
data = block_missing(data[np.newaxis,...],factor=0.1, block_len=3, block_width=3)
data = data[0]
data

2025-07-03 19:45:54 [WARNING]: hit_rate=16.666666666666668 > 1


array([[        nan,         nan,         nan, ...,         nan,         nan,         nan],
       [        nan,         nan,         nan, ...,         nan,         nan,         nan],
       [        nan,         nan,         nan, ...,         nan,         nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,         nan,  7.22265962],
       [-2.01277392,         nan,         nan, ...,         nan,         nan,  6.90596591],
       [-0.23049405,  4.89620653, 11.67355353, ...,         nan,  4.23932052,  6.62162548]])

In [11]:
data_imputed = impute(data, cg, model_params)
data_imputed

array([[-2.5680993 , -1.0196397 ,  4.5346837 , ..., -1.        ,  4.2393208 ,  7.0566187 ],
       [-3.5071251 , -1.0196397 ,  4.4862537 , ..., -1.        ,  4.2393208 ,  7.0566206 ],
       [-3.9279313 , -1.0196397 ,  4.505824  , ..., -1.        ,  4.2393208 ,  7.0566244 ],
       ...,
       [-3.9150765 ,  4.3270555 , 11.324441  , ..., -1.        ,  4.2393208 ,  7.2226596 ],
       [-2.012774  ,  4.60882   , 12.077079  , ..., -1.        ,  4.2393208 ,  6.905966  ],
       [-0.23049405,  4.8962064 , 11.673553  , ..., -1.        ,  4.2393208 ,  6.6216254 ]], dtype=float32)

In [1]:
# test_miracle.py
import numpy as np
from baseline import miracle_impu

# 创建测试数据
test_data = np.random.randn(100, 10)
test_data[np.random.random((100, 10)) < 0.1] = np.nan

print("测试数据形状:", test_data.shape)
print("缺失值数量:", np.isnan(test_data).sum())

try:
    result = miracle_impu(test_data)
    print("MIRACLE结果形状:", result.shape if result is not None else "None")
    print("MIRACLE结果类型:", type(result))
    
    if result is not None:
        print("填补后缺失值:", np.isnan(result).sum())
    else:
        print("❌ MIRACLE返回了None")
        
except Exception as e:
    print(f"❌ MIRACLE测试失败: {e}")
    import traceback
    traceback.print_exc()

测试数据形状: (100, 10)
缺失值数量: 113


2025-07-11 14:06:50.758518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-11 14:06:54.021479: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term
_____________1______________________


2025-07-11 14:07:09.701656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11050 MB memory:  -> device: 0, name: NVIDIA TITAN Xp, pci bus id: 0000:02:00.0, compute capability: 6.1
2025-07-11 14:07:09.702417: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 11050 MB memory:  -> device: 1, name: NVIDIA TITAN Xp, pci bus id: 0000:03:00.0, compute capability: 6.1
2025-07-11 14:07:09.703024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 11050 MB memory:  -> device: 2, name: NVIDIA TITAN Xp, pci bus id: 0000:82:00.0, compute capability: 6.1
2025-07-11 14:07:09.703622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 11050 MB memory:  -> device: 3, name: NVIDIA TITAN Xp, pci bus id: 0000:83:00.0, comp

MIRACLE结果形状: (100, 10)
MIRACLE结果类型: <class 'numpy.ndarray'>
填补后缺失值: 0
